In [6]:
import numpy as np
import scipy.signal as scp

# Chcel by som implementovať ešte max pool, avg pool, dense layer, softmax, sigmoid, cross-entropy

In [22]:
class Conv:
    def __init__(self, input_shape, kernel_size, kernel_shape = (3, 3), stride = 1, padding = 0) -> None:
        input_depth, input_height, input_width = input_shape
        self.kernel_size = kernel_size
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.kernel = kernel_shape
        self.kernel_shape = (kernel_size, input_depth, *kernel_shape)
        self.output_shape = (kernel_size, int((input_height - kernel_shape[0] + 2 * padding) / stride + 1), int((input_width - kernel_shape[0] + 2 * padding) / stride + 1))
        self.biases = np.random.rand(*self.output_shape)
        self.kernels = np.random.rand(*self.kernel_shape)
        self.stride = stride
        self.zero_padding = padding
        
    
    def forward(self, input):
        self.output = np.copy(self.biases)
        
        for i in range(self.kernel_size):
            for j in range(self.input_depth):
                padded_input = np.pad(input[j], self.zero_padding)
                windowed_input = np.lib.stride_tricks.sliding_window_view(padded_input, window_shape = self.kernel)[::self.stride, ::self.stride]
                for index_row, windowed_steps in enumerate(windowed_input):
                    for index_col, window in enumerate(windowed_steps):
                        self.output[i][index_row][index_col] += np.sum(np.multiply(window, self.kernels[i, j]))
        
        return self.output
    
    # def full_convolution(self, input):
    #     output = np.zeros((input.shape[0], input.shape[1] + self.kernel[0] - 1, input.shape[2] + self.kernel[1] - 1))
    #     for i in range(self.kernel_size):
    #         for j in range(self.input_depth):
    #             padded = np.pad(input[j], self.kernel[0] - 1)
    #             windowed_input = np.lib.stride_tricks.sliding_window_view(padded, window_shape = self.kernel)[::self.stride, ::self.stride]
    #             for index_row, windowed_steps in enumerate(windowed_input):
    #                 for index_col, window in enumerate(windowed_steps):
    #                     output[i][index_row][index_col] += np.sum(np.multiply(window, self.kernels[i, j]))
    #     return output
    
    def backward(self, output_grads, learning_rate) -> None:
        kernel_grads = np.zeros(self.kernels.shape)
        input_grads = np.zeros(self.input.shape)
        
        for i in range(self.kernel_size):
            padded_output_grands = np.pad(output_grads[i], self.kernel[0] - 1)
            windowed_output_grads = np.lib.stride_tricks.sliding_window_view(padded_output_grands, window_shape = self.kernel)[::self.stride, ::self.stride]
            for j in range(self.input_depth):
                padded_input = np.pad(self.input[j], self.zero_padding)
                windowed_input = np.lib.stride_tricks.sliding_window_view(padded_input, window_shape = self.kernel)[::self.stride, ::self.stride]
                for index_row, windowed_steps in enumerate(windowed_input):
                    for index_col, window in enumerate(windowed_steps):
                        kernel_grads[i, j] += np.sum(np.multiply(window, self.output_grads[i]))
                for index_row, windowed_steps in enumerate(windowed_output_grads):
                    for index_col, window in enumerate(windowed_steps):
                        input_grads[j] += np.sum(np.multiply(window, self.kernels[i, j]))
        
        self.kernels -= learning_rate * kernel_grads
        self.biases -= learning_rate * output_grands
        return input_grads
        

In [30]:
conn1 = Conv((1, 6, 6), 3, (3, 3), 1, 1)
arr = np.array([[[1, 1, -1, 1, -1, 1],
              [1, 1, 1, 1, 1, -1],
              [1, -1, 1, -1, 1, -1],
              [1, 1, 1, 1, 1, 1],
              [1, 1, -1, 1, 1, 1],
              [1, -1, 1, -1, 1, 1]]])

conn1.forward(arr)



array([[[2.86230373, 2.50542134, 2.49458662, 1.01776822, 1.26841541,
         0.20424038],
        [3.76330592, 2.99706462, 3.60759149, 2.74987823, 2.43753587,
         0.6129791 ],
        [2.72068352, 3.46144427, 2.38518117, 3.49465223, 1.24278969,
         0.95643577],
        [1.54107711, 4.53964175, 2.2953877 , 4.40118631, 2.96510085,
         3.40824206],
        [3.64788321, 4.19592497, 2.0339335 , 3.26320221, 4.94351762,
         3.85197486],
        [1.74072343, 0.85131494, 0.92254644, 1.60260985, 2.60276702,
         3.02875193]],

       [[2.15564003, 3.21147003, 1.99124253, 1.76339274, 1.32760018,
         0.30887088],
        [1.77417038, 2.97031062, 0.88071491, 2.44986342, 1.26273927,
         1.56539838],
        [3.03594679, 3.14008597, 2.53333004, 3.35681572, 2.00062388,
         0.85350464],
        [2.477827  , 2.46007052, 2.82245634, 2.16746758, 3.12965168,
         2.65621944],
        [1.71051044, 3.38964754, 0.90151582, 2.2142837 , 3.70470833,
         3.52066916

In [88]:
class MaxPool():
    def __init__(self, input_shape, depth):
        input_depth, input_height, input_width = input_shape
        self.input_shape = input_shape
        self.depth = depth
        self.input_depth = input_depth
        self.input_height = input_height
        self.input_width = input_width
        self.output_shape = (input_depth, int(input_height / depth), int(input_width / depth))
        self.kernel = (depth, depth)
        self.indices = np.zeros(self.output_shape)
    
    def forward(self, input):
        help_indices = np.arange(0, self.input_height * self.input_width).reshape((self.input_height, self.input_width))
        output = np.zeros(self.output_shape)
        
        for i in range(self.input_depth):
            windowed_input = np.lib.stride_tricks.sliding_window_view(input[i], window_shape = self.kernel)[::2, ::2]
            windowed_indices = np.lib.stride_tricks.sliding_window_view(help_indices, window_shape = self.kernel)[::2, ::2]
            for index_row, (w_s, w_i_s) in enumerate(zip(windowed_input, windowed_indices)):
                for index_col, (window, idices_step) in enumerate(zip(w_s, w_i_s)):
                    output[i][index_row][index_col] = window.max()
                    self.indices[i][index_row][index_col] = idices_step.reshape(-1)[window.argmax()]
        return output
            
        
    
    def backward(self):
        pass

In [90]:
maxpool = MaxPool((1, 4, 4), 2)

maxpool.forward([[[1, 2, 3, 4], 
                  [2, 3, 4, 5], 
                  [7, 10, 4, 2], 
                  [9, 8, 3, 2]]])

maxpool.indices

array([[[ 5.,  7.],
        [ 9., 10.]]])

In [100]:
class AvgPool():
    def __init__(self, input_shape, depth):
        input_depth, input_height, input_width = input_shape
        self.input_shape = input_shape
        self.depth = depth
        self.input_depth = input_depth
        self.input_height = input_height
        self.input_width = input_width
        self.output_shape = (input_depth, int(input_height / depth), int(input_width / depth))
        self.kernel = (depth, depth)
    
    def forward(self, input):
        output = np.zeros(self.output_shape)
        
        for i in range(self.input_depth):
            windowed_input = np.lib.stride_tricks.sliding_window_view(input[i], window_shape = self.kernel)[::2, ::2]
            for index_row, w_s in enumerate(windowed_input):
                for index_col, window in enumerate(w_s):
                    output[i][index_row][index_col] = np.average(window)
        return output
    
    def backward(self):
        pass

In [101]:
maxpool = AvgPool((1, 4, 4), 2)

maxpool.forward([[[1, 2, 3, 4], 
                  [2, 3, 4, 5], 
                  [7, 10, 4, 2], 
                  [9, 8, 3, 2]]])


array([[[2.  , 4.  ],
        [8.5 , 2.75]]])

In [120]:
class Sigmoid():
    def forward(self, input):
        return 1 / (1 + np.exp(np.negative(input)))
    
    def backward(self):
        pass

In [154]:
class ReLU():
    def forward(self, input):
        return np.maximum(input, 0)
    
    def backward(self):
        pass

In [155]:
relu = ReLU()

relu.forward([[[1, -2, 3, 4], 
              [2, 3, 4, 5], 
              [7, 10, 4, 2], 
              [9, 8, 3, 2]]])

array([[[ 1,  0,  3,  4],
        [ 2,  3,  4,  5],
        [ 7, 10,  4,  2],
        [ 9,  8,  3,  2]]])

In [156]:
class Reshape():
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
    
    def forward(self, input):
        return input.reshape(self.output_shape)
    
    def backward(self, output_grad):
        return output_grad.reshape(self.input_shape)

In [144]:
class Dense():
    def __init__(self, input_shape, output_shape):
        self.weights = np.random.rand(output_shape, input_shape)
        self.biases = np.random.rand(output_shape, 1)
        
    
    def forward(self, input):
        return self.weights @ input + self.biases
    
    def backward(self):
        pass

In [146]:
dense = Dense(6, 10)

dense.forward([[[1, 1, 1, 1, 1, 1],
                [2, 2, 2, 2, 2, 2],
                [3, 3, 3, 3, 3, 3],
                [4, 4, 4, 4, 4, 4],
                [5, 5, 5, 5, 5, 5],
                [6, 6, 6, 6, 6, 6]]])


array([[[11.60498984, 11.60498984, 11.60498984, 11.60498984,
         11.60498984, 11.60498984],
        [ 9.91985237,  9.91985237,  9.91985237,  9.91985237,
          9.91985237,  9.91985237],
        [ 8.26242806,  8.26242806,  8.26242806,  8.26242806,
          8.26242806,  8.26242806],
        [12.40416641, 12.40416641, 12.40416641, 12.40416641,
         12.40416641, 12.40416641],
        [11.96120017, 11.96120017, 11.96120017, 11.96120017,
         11.96120017, 11.96120017],
        [ 4.91735791,  4.91735791,  4.91735791,  4.91735791,
          4.91735791,  4.91735791],
        [11.09922011, 11.09922011, 11.09922011, 11.09922011,
         11.09922011, 11.09922011],
        [14.10072339, 14.10072339, 14.10072339, 14.10072339,
         14.10072339, 14.10072339],
        [14.96192524, 14.96192524, 14.96192524, 14.96192524,
         14.96192524, 14.96192524],
        [12.96648099, 12.96648099, 12.96648099, 12.96648099,
         12.96648099, 12.96648099]]])

In [ ]:
def cross_entropy_loss(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred) + (1 - y_pred) * np.log(1 - y_pred))

In [ ]:
def softmax(X);
    exps = np.exp(X)
    return exps / np.sum(exps)

In [34]:
np.array([[2, 3], [4, 5]]).max()

5

In [42]:
np.array([[2, 3], [4, 5]]).argmax()

np.arange(0, 16).reshape((4, 4))

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])